### importing the data

In [ ]:
import pandas as pd
df = pd.read_csv("arxiv_train.csv")
#exploring the data
print(df.head()) 
print(df.info()) 
print(df['label'].value_counts())

   Unnamed: 0                                           abstract     label
0       31716    Automatic meeting analysis is an essential f...      eess
1       89533    We propose a protocol to encode classical bi...  quant-ph
2       82700    A number of physically intuitive results for...  quant-ph
3       78830    In the last decade rare-earth hexaborides ha...   physics
4       94948    We introduce the weak barycenter of a family...      stat
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  80000 non-null  int64 
 1   abstract    80000 non-null  object
 2   label       80000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB
None
label
stat        8055
eess        8052
physics     8023
hep-ph      8010
cs          8005
quant-ph    7987
astro-ph    7987
hep-th      7981
math        7958
cond-mat    7942
Name: count